In [2]:
from typing import Generator, Iterable
import datetime
import tqdm
import decimal
import numpy as np
import pandas as pd
import plotly.express as px

import sqlite3
from google.cloud import bigquery

In [3]:
bq = bigquery.Client()
inmem_db = sqlite3.connect(":memory:")

SHIFTING_PRECISION_FACTOR = 10**8

In [4]:

query = '''DROP TABLE IF EXISTS order_book;'''
inmem_db.execute(query)

In [5]:
query = '''
CREATE TABLE order_book (
    price BIGINT,
    side VARCHAR(8),
    size BIGINT,
    PRIMARY KEY (price)
);
'''
cur = inmem_db.execute(query)


In [6]:
MAX_PROCESSING_TIME = 3 * 60


__DATETIME_FORMAT = r'%Y-%m-%d %H-%M'

In [7]:
symbol = 'BTC-USD'

In [8]:
start_execution_timestamp = '2022-06-08 08:47:38'
start_execution_datetime = datetime.datetime.fromisoformat(start_execution_timestamp)

In [9]:

def find_initial_snapshot_datetime(symbol: str,
                                    start_execution_datetime: datetime.datetime) -> datetime.datetime:

    query = '''
    SELECT MAX(`timestamp`) AS `timestamp`
    FROM `trading_terminal_poc.coinbase_snapshot_timestamp`
    WHERE `symbol` = @symbol
            AND DATETIME(`timestamp`) BETWEEN @start_datetime AND @end_datetime
    '''

    start_datetime = (start_execution_datetime - datetime.timedelta(days=1))
    end_datetime = start_execution_datetime

    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter('symbol', 'STRING', symbol),
            bigquery.ScalarQueryParameter('start_datetime', 'DATETIME', start_datetime),
            bigquery.ScalarQueryParameter('end_datetime', 'DATETIME', end_datetime)
        ]
    )

    query_job = bq.query(query, job_config=job_config)

    result = query_job.result()
    if result.total_rows > 0:
        initial_snapshot_datetime = next(result).get('timestamp')
        return initial_snapshot_datetime
    else:
        raise NotImplementedError


In [10]:
initial_snapshot_datetime = find_initial_snapshot_datetime(symbol, start_execution_datetime)
initial_snapshot_datetime

datetime.datetime(2022, 6, 8, 8, 36, 37, 436280)

In [11]:
def query_raw_l2_order_book(symbol: str,
						start_datetime: datetime.datetime, 
						end_datetime: datetime.datetime) -> bigquery.table.RowIterator:
	query = '''
	WITH l2_order_book AS (
		SELECT *
		FROM `trading_terminal_poc.coinbase_raw_l2_order_book`
		WHERE `symbol` = @symbol AND
				`timestamp` BETWEEN @start_datetime AND @end_datetime 
	),
	norm_l2 AS (
		SELECT DATE_TRUNC(`timestamp`, SECOND) AS `timestamp_norm`,
				price,
				MAX(`timestamp`) AS `timestamp`
		FROM l2_order_book
		GROUP BY 1, 2
	)
	SELECT norm_l2.timestamp_norm,
			l2_order_book.side,
			norm_l2.price,
			l2_order_book.`size` AS `size`
	FROM norm_l2
	INNER JOIN l2_order_book USING (`timestamp`, price)
	ORDER BY norm_l2.timestamp_norm
	'''

	job_config = bigquery.QueryJobConfig(
		query_parameters=[
			bigquery.ScalarQueryParameter('symbol', 'STRING', symbol),
			bigquery.ScalarQueryParameter('start_datetime', 'DATETIME', start_datetime),
			bigquery.ScalarQueryParameter('end_datetime', 'DATETIME', end_datetime)
		]
	)

	query_job = bq.query(query, job_config=job_config)
	result = query_job.result()

	return result

In [12]:
start_datetime = initial_snapshot_datetime 
end_datetime = start_execution_datetime + datetime.timedelta(seconds=MAX_PROCESSING_TIME)
result = query_raw_l2_order_book(symbol, start_datetime, end_datetime)
df = result.to_dataframe()
df

,timestamp_norm,side,price,size
0,2022-06-08 08:36:37,buy,30474.93000000000000000000000000000000000000,0E-38
1,2022-06-08 08:36:37,buy,30474.95000000000000000000000000000000000000,0.05000000000000000000000000000000000000
2,2022-06-08 08:36:37,buy,30474.80000000000000000000000000000000000000,0E-38
3,2022-06-08 08:36:37,sell,30478.58000000000000000000000000000000000000,0.02105982000000000000000000000000000000
4,2022-06-08 08:36:37,buy,30474.96000000000000000000000000000000000000,0.00252962000000000000000000000000000000
...,...,...,...,...
154267,2022-06-08 08:50:37,sell,30490.46000000000000000000000000000000000000,0.10000000000000000000000000000000000000
154268,2022-06-08 08:50:37,sell,30481.79000000000000000000000000000000000000,2.01460000000000000000000000000000000000
154269,2022-06-08 08:50:37,sell,30470.79000000000000000000000000000000000000,0E-38
154270,2022-06-08 08:50:37,sell,30468.18000000000000000000000000000000000000,0.23500000000000000000000000000000000000


In [13]:
df = df[(df['timestamp_norm'] == '2022-06-08 08:36:37')]
df[['price', 'size']] = df[['price', 'size']].astype(np.float64)
df = df[(df['price'] > 10000) & (df['price'] < 60000)]
df

/Users/anythingth2/development/.python_virtual_env/cft_env/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,timestamp_norm,side,price,size
0,2022-06-08 08:36:37,buy,30474.93,0.000000
1,2022-06-08 08:36:37,buy,30474.95,0.050000
2,2022-06-08 08:36:37,buy,30474.80,0.000000
3,2022-06-08 08:36:37,sell,30478.58,0.021060
4,2022-06-08 08:36:37,buy,30474.96,0.002530
...,...,...,...,...
41387,2022-06-08 08:36:37,sell,30481.44,0.039743
41388,2022-06-08 08:36:37,buy,30472.57,0.000000
41389,2022-06-08 08:36:37,buy,30475.00,0.002530
41390,2022-06-08 08:36:37,buy,30475.04,0.002530


In [14]:
df.describe()

,price,size
count,31069.000000,31069.000000
mean,34094.298716,0.522800
std,11757.640773,6.374318
min,10000.010000,0.000000
25%,25496.820000,0.000738
50%,31610.000000,0.004182
75%,43490.640000,0.044577
max,59999.990000,402.753675


In [15]:
px.histogram(df,
        x='size',
        y='price',
        orientation='h',
        color='side',
        histfunc='sum'
        )

In [16]:
x = decimal.Decimal('0.00000001')
x = (x * SHIFTING_PRECISION_FACTOR)
int(x)

1

In [17]:
def query_order_book(from_price: decimal.Decimal = None,
                        to_price: decimal.Decimal = None,
                        coerce_float: bool = False
                    ) -> pd.DataFrame:
    
    if from_price is not None:
        from_price = int(from_price * SHIFTING_PRECISION_FACTOR)
    
    if to_price is not None:
        to_price = int(to_price * SHIFTING_PRECISION_FACTOR)
        
    if from_price is not None and to_price is not None:
        query = '''
        SELECT *
        FROM order_book
        WHERE from_price > ? AND to_price < ?
        ORDER BY price
        '''

        params = (from_price, to_price)

    elif from_price is not None:
        query = '''
        SELECT *
        FROM order_book
        WHERE from_price > ? 
        ORDER BY price
        '''

        params = (from_price, )

    elif to_price is not None:
        query = '''
        SELECT *
        FROM order_book
        WHERE to_price < ?
        ORDER BY price
        '''

        params = (to_price, )
    else:
        query = '''
        SELECT *
        FROM order_book
        ORDER BY price
        '''
        params = None

    df = pd.read_sql(query, inmem_db, params=params, coerce_float=coerce_float)
    df.loc[:, ['price', 'size']] /= SHIFTING_PRECISION_FACTOR


    return df

df = query_order_book()
df

,price,side,size


In [18]:
def upsert_order_book(price: decimal.Decimal, side: str, size: decimal.Decimal):
    query = '''
    INSERT INTO order_book(price, side, size)
        VALUES (?, ?, ?)
        ON CONFLICT(price)
            DO UPDATE SET side = ?,
                            size = ?
    '''
    price = int(price * SHIFTING_PRECISION_FACTOR)
    size = int(size * SHIFTING_PRECISION_FACTOR)

    params = (price, side, size, side, size)

    cursor = inmem_db.cursor()
    cursor.execute(query, params)

In [19]:
def delete_order_book(price: decimal.Decimal):
    query = '''
    DELETE FROM order_book
    WHERE price = ?
    '''

    price = int(price * SHIFTING_PRECISION_FACTOR)
    params = (price, )

    cursor = inmem_db.cursor()
    cursor.execute(query, params)

In [27]:
def truncate_order_book():
    query = '''
    DELETE FROM order_book
    '''

    cursor = inmem_db.cursor()
    cursor.execute(query)

In [20]:
def query_median_price() -> float:
    query = '''
    SELECT MAX(price) AS buy_price
    FROM order_book
    WHERE side = "buy"
    '''
    buy_price_df = pd.read_sql(query, inmem_db)
    buy_price = buy_price_df.iloc[0]['buy_price']

    query = '''
    SELECT MIN(price) AS sell_price
    FROM order_book
    WHERE side = "sell"
    '''
    sell_price_df = pd.read_sql(query, inmem_db)
    sell_price = sell_price_df.iloc[0]['sell_price']

    median_price = (buy_price + sell_price) / 2
    median_price /= SHIFTING_PRECISION_FACTOR 
    return median_price

In [36]:
def query_snapshot_timestamp(symbol: str, 
                                start_datetime: datetime.datetime, 
                                end_datetime: datetime.datetime) -> pd.Series:
    query = '''
    SELECT `timestamp`
    FROM `trading_terminal_poc.coinbase_snapshot_timestamp`
    WHERE symbol = @symbol AND
            `timestamp` BETWEEN @start_datetime AND @end_datetime 
    ORDER BY `timestamp`
    '''

    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter('symbol', 'STRING', symbol),
			bigquery.ScalarQueryParameter('start_datetime', 'DATETIME', start_datetime),
			bigquery.ScalarQueryParameter('end_datetime', 'DATETIME', end_datetime)
        ]
    )

    query_job = bq.query(query, job_config=job_config)
    return query_job.to_dataframe()['timestamp']

In [62]:

class MarketSimulator:
    def __init__(self,
                    symbol: str,
                    start_datetime: datetime.datetime,
                    end_datetime: datetime.datetime
                ):
        self.symbol = symbol
        self.start_datetime = start_datetime
        self.end_datetime = end_datetime

    def pre_simulate(self):

        initial_snapshot_datetime = find_initial_snapshot_datetime(self.symbol, self.start_datetime)
        order_book_result = query_raw_l2_order_book(symbol, initial_snapshot_datetime, self.start_datetime)

        # Iterate over all record of order book in either same timestamp and next timestamp
        for row in tqdm.tqdm(order_book_result, total=order_book_result.total_rows):
            price = row.get('price')
            side = row.get('side')
            size = row.get('size')
            if size > 0:
                
                upsert_order_book(price=price,
                                    side=side,
                                    size=size
                )
            else:
                delete_order_book(price=price)

    def simulate(self) -> Iterable[datetime.datetime]:

        self.pre_simulate()


        snapshot_timestamps = query_snapshot_timestamp(self.symbol, self.start_datetime, self.end_datetime)
        order_book_result = query_raw_l2_order_book(symbol, self.start_datetime, self.end_datetime)

        current_datetime = None
        # Iterate over all record of order book in either same timestamp and next timestamp
        for row in tqdm.tqdm(order_book_result, total=order_book_result.total_rows):
            order_book_datetime = row.get('timestamp_norm')

            # Check new timeframe
            if current_datetime is not None:
                # Yield if start new timeframe
                if order_book_datetime > current_datetime:
                    yield current_datetime
                    current_datetime = order_book_datetime
            else:
                current_datetime = order_book_datetime

            # Does cursor reach new snapshot ?
            if len(snapshot_timestamps) > 0 and order_book_datetime > snapshot_timestamps.min():
                snapshot_timestamps.pop(0)
                truncate_order_book()

            price = row.get('price')
            side = row.get('side')
            size = row.get('size')
            if size > 0:
                
                upsert_order_book(price=price,
                                    side=side,
                                    size=size
                )
            else:
                delete_order_book(price=price)
        
        yield current_datetime
    
    def simulate_with_granularity(self, granularity: int) -> Iterable[datetime.datetime]:
        market_iterator = self.simulate()
        while True:
            for _ in range(granularity):
                current_datetime = next(market_iterator)
            yield current_datetime
        

    def plot_bid_ask(self):

        median_price = query_median_price()
        order_book_df = query_order_book(coerce_float=True)
        order_book_df = order_book_df.loc[(order_book_df['price'] > median_price / 2) & (order_book_df['price'] < median_price * 3 / 2)]
        
        return px.histogram(order_book_df,
            x='size',
            y='price',
            orientation='h',
            color='side',
            histfunc='sum'
            )
            

In [63]:
end_datetime = start_execution_datetime + datetime.timedelta(seconds=MAX_PROCESSING_TIME)

market_simulator = MarketSimulator(symbol=symbol,
                                    start_datetime=start_execution_datetime,
                                    end_datetime=end_datetime
                                )
start_execution_datetime, end_datetime

(datetime.datetime(2022, 6, 8, 8, 47, 38),
 datetime.datetime(2022, 6, 8, 8, 50, 38))

In [64]:
iterable = market_simulator.simulate_with_granularity(granularity=5)

In [70]:
next(iterable)

 10%|▉         | 2317/23624 [00:11<01:25, 249.92it/s]

datetime.datetime(2022, 6, 8, 8, 48, 7)

 10%|▉         | 2317/23624 [00:24<01:25, 249.92it/s]

In [51]:
market_simulator.plot_bid_ask()

 27%|██▋       | 41160/154272 [00:23<00:14, 7756.14it/s]

In [62]:
next(iterable)
market_simulator.plot_bid_ask()

 28%|██▊       | 42710/154272 [01:03<21:51, 85.04it/s]